In [29]:
#Installing necessary packages via terminal:
# pip install pandas azure-storage-blob python-dotenv

In [30]:
#Importing necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os


In [31]:
#Data Extraction
try:
    data = pd.read_csv(r'..\dataset\zipco_transaction.csv')
    print("Data Extracted successfully.")
except Exception as e:
    print(f"Error extracting data: {e}")

Data Extracted successfully.


In [32]:
data.head(5)  # Display the first few rows of the data

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [33]:
data.info()  # Display information about the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [34]:
#Data Cleaning and Transformation
#Remove duplicates
data.drop_duplicates(inplace=True)

In [35]:
#Handle missing values (filling missing values with the mean or median of the column)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace=True)

In [36]:
#Handle missing values (fill missing string/object values with 'Unknown')
string_columns = data.select_dtypes(include=['object']).columns
for col in string_columns:
    data.fillna({col:'Unknown'}, inplace=True)

In [37]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

In [38]:
data.info()  # Display information about the DataFrame

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [54]:
#create the product table
products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'product_id'
products = products.reset_index()


In [55]:
products.head(5)  # Display the first few rows of the products table

,product_id,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni


In [41]:
#create the staff table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'staff_id'
staff = staff.reset_index()

In [42]:
staff.head(5)  # Display the first few rows of the staffs table

,staff_id,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org


In [56]:
customers = data[['CustomerName', 'CustomerAddress','Customer_PhoneNumber', 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'customer_id'
customers = customers.reset_index()

In [57]:
customers.head(5)  # Display the first few rows of the customers table

,customer_id,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net


In [58]:
#Create Transactions tables
transactions = data.merge(product, on=['ProductName'], how='left') \
                .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left') \
                .merge(customer, on=['CustomerName', 'CustomerAddress','Customer_PhoneNumber', 'CustomerEmail'], how='left') \
        
transactions.index.name = 'transaction_id'
transactions = transactions.reset_index() \
[['transaction_id', 'product_id', 'customer_id', 'staff_id', 'Date', 'Quantity', 'UnitPrice', \
       'StoreLocation', 'PaymentType', 'PromotionApplied', 'Weather','Temperature', 'StaffPerformanceRating', 'CustomerFeedback', \
       'DeliveryTime_min', 'OrderType','DayOfWeek', 'TotalSales']]
    
    


In [59]:
transactions.head(10) # Display the first few rows of the transactions table

,transaction_id,product_id,customer_id,staff_id,Date,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,CustomerFeedback,DeliveryTime_min,OrderType,DayOfWeek,TotalSales
0,0,0,0,0,2023-01-01 00:00:00,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,Neutral,30,In Store,Sunday,25.064608
1,1,1,1,1,2023-01-01 01:00:00,1,7.083070,South,Cash,False,Rainy,23.549497,Average,Positive,33,In Store,Sunday,7.083070
2,2,2,2,2,2023-01-01 02:00:00,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,Unknown,43,Phone Order,Sunday,33.680321
3,3,3,3,3,2023-01-01 03:00:00,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,Positive,32,Online Order,Sunday,14.629647
4,4,4,4,4,2023-01-01 04:00:00,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,Neutral,58,In Store,Sunday,7.577727
5,5,1,5,5,2023-01-01 05:00:00,2,14.568156,West,Cash,True,Rainy,27.154342,Poor,Neutral,30,Phone Order,Sunday,29.136313
6,6,1,6,6,2023-01-01 06:00:00,3,16.114719,North,Credit Card,False,Cloudy,29.201028,Excellent,Unknown,15,In Store,Sunday,48.344156
7,7,3,7,7,2023-01-01 07:00:00,5,12.949873,West,Online,False,Sunny,27.154342,Average,Positive,16,Online Order,Sunday,64.749364
8,8,3,8,8,2023-01-01 08:00:00,4,17.435266,East,Cash,True,Rainy,27.154342,Poor,Neutral,29,Phone Order,Sunday,69.741064
9,9,5,9,9,2023-01-01 09:00:00,2,19.868948,West,Credit Card,False,Rainy,20.422870,Average,Neutral,53,In Store,Sunday,39.737897


In [60]:
transactions.info()  # Display information about the DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          1000 non-null   int64         
 1   product_id              1000 non-null   int64         
 2   customer_id             1000 non-null   int64         
 3   staff_id                1000 non-null   int64         
 4   Date                    1000 non-null   datetime64[ns]
 5   Quantity                1000 non-null   int64         
 6   UnitPrice               1000 non-null   float64       
 7   StoreLocation           1000 non-null   object        
 8   PaymentType             1000 non-null   object        
 9   PromotionApplied        1000 non-null   bool          
 10  Weather                 1000 non-null   object        
 11  Temperature             1000 non-null   float64       
 12  StaffPerformanceRating  1000 non-null   object   

In [61]:
#Save data to CSV files
data.to_csv(r'..\dataset\cleaned_data.csv', index=False)   
products.to_csv(r'..\dataset\products.csv', index=False)
staff.to_csv(r'..\dataset\staff.csv', index=False)
customers.to_csv(r'..\dataset\customers.csv', index=False)
transactions.to_csv(r'..\dataset\transactions.csv', index=False)

In [52]:
#Data Loading
#Load the enviroment variables from the .env files
load_dotenv()

connect_str = os.getenv("AZURE_CONNECTION_STRING_VALUE")
container_name = os.getenv("CONTAINER_NAME")


In [62]:
#Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

#Load data to Azure Blob Storage
files = [
    (data, 'rawdata/cleaned_zipco_transaction_data.csv'),
    (products, 'cleaneddata/products.csv'),
    (staff, 'cleaneddata/staff.csv'),
    (customers, 'cleaneddata/customers.csv'),
    (transactions, 'cleaneddata/transactions.csv')
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f"Uploaded {blob_name} to Azure Blob Storage successfully.")

Uploaded rawdata/cleaned_zipco_transaction_data.csv to Azure Blob Storage successfully.
Uploaded cleaneddata/products.csv to Azure Blob Storage successfully.
Uploaded cleaneddata/staff.csv to Azure Blob Storage successfully.
Uploaded cleaneddata/customers.csv to Azure Blob Storage successfully.
Uploaded cleaneddata/transactions.csv to Azure Blob Storage successfully.
